# HW 1: Corporate Bond Pricing (due by 9.20 Fri) ffafwefwefw

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [168]:
import numpy as np

In [169]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [170]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.038363556534703, 10.0)

In [176]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    value=np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)),np.exp(-rf_rate*mat)))
    return value

# Call function
corp_bond(mat=mat, def_rate=def_rate, rf_rate=rf_rate, recovery=recovery, n_sample=n_sample)

# Find the mean and std by calling the function 100 times. 

nsim=100
monc=[]

for ii in range(nsim):
    monc.append(corp_bond(mat=mat, def_rate=def_rate, rf_rate=rf_rate, recovery=recovery, n_sample=n_sample))


print(np.mean(monc),np.std(monc))

0.44057256249533977 0.0022865863979439735


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [177]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=np.int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [187]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    if(antithetic):
        U = np.random.uniform(size=np.int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        
        value = np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)),np.exp(-rf_rate*mat)))
        
    if(mean_match):
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean()
        
        value = np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)),np.exp(-rf_rate*mat)))
          
    return value

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

nsim=100
monc=[]

print('antithetic')
for ii in range(nsim):
    monc.append(corp_bond_cv(mat=mat, def_rate=def_rate, rf_rate=rf_rate, recovery=recovery, n_sample=n_sample, antithetic=True, mean_match=False))
print(np.mean(monc),np.std(monc))

print('mean_match')
for ii in range(nsim):
    monc.append(corp_bond_cv(mat=mat, def_rate=def_rate, rf_rate=rf_rate, recovery=recovery, n_sample=n_sample, antithetic=False, mean_match=True))
print(np.mean(monc),np.std(monc))

antithetic
0.4407310717801043 0.0016388164976305329
mean_match
0.44064648911690013 0.0016382770415532294


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [184]:
### Put the analytic expression for the corporate bond price

def analytic(mat=1, def_rate=1, rf_rate=1, recovery=1):
    bond = np.exp(-(rf_rate+def_rate)*mat)
    p = bond + recovery*def_rate/(rf_rate+def_rate)*(1-bond)
    
    return p

analytic(mat=mat, def_rate=def_rate, rf_rate=rf_rate, recovery=recovery)

0.44040907156462505

From this we see that by using the Monte Carlo methods we get results very close to the analytical value. 